In [1]:
from collections import namedtuple

In [2]:
Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price) -> None:
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self) -> float:
        return self.quantity * self.price


class Order:
    """上下文"""

    def __init__(self, customer, cart, promotion=None) -> None: 
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self) -> float:
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart) 
        return self.__total
    
    def due(self) -> float:
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self) -> str:
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'

In [3]:
promos = []

def promotion(func):
    promos.append(func)
    return func

@promotion
def fidelity_promo(order) -> float:
    """为积分为1000或以上的顾客提供5%折扣"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0 

@promotion
def bulk_item_promo(order) -> float:
    """单个商品为20个或以上时提供10%折扣"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order_promo(order) -> float:
    """订单中的不同商品达到10个或以上时提供7%折扣"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [4]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)
]

In [5]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [6]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [7]:
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [8]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [9]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

In [10]:
def best_promo(order) -> float:
    """选择可用的最佳折扣"""
    return max(promo(order) for promo in promos)

In [11]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [12]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [13]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>